In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger("PIL").setLevel(logging.ERROR)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
base_directoryectory= "/workspaces/machinelearning/animaldata"
output_directoryectory= "/workspaces/machinelearning/output"

train_directoryectory= os.path.join(output_directoryectory, "train")
os.mkdirectory(train_directoryectory)

test_directoryectory= os.path.join(output_directoryectory, "test")
os.mkdirectory(test_directoryectory)

validation_directoryectory= os.path.join(output_directoryectory, "validation")
os.mkdirectory(validation_directoryectory)

train_cats_directory = os.path.join(train_directory, 'cats')
os.mkdirectory(train_cats_directory)
train_dogs_directory = os.path.join(train_directory, 'dogs')
os.mkdirectory(train_dogs_directory)

test_cats_directory = os.path.join(test_directory, 'cats')
os.mkdirectory(test_cats_directory)
test_dogs_directory = os.path.join(test_directory, 'dogs')
os.mkdirectory(test_dogs_directory)

validation_cats_directory = os.path.join(validation_directory, 'cats')
os.mkdirectory(validation_cats_directory)
validation_dogs_directory = os.path.join(validation_directory, 'dogs')
os.mkdirectory(validation_dogs_directory)


cat_directory= "/workspaces/machinelearning/animaldata/Cat"
dog_directory= "/workspaces/machinelearning/animaldata/Dog"

fnamestrain= ["{}.jpg". format(i) for i in range(8750)]
fnamestest= ["{}.jpg". format(i) for i in range(8750, 11250)]
fnamesval= ["{}.jpg". format(i) for i in range(11250, 12499)]

def Calc(src, dst): 
    if not os.path.exists(src):
        return 
    if os.path.getsize(src)==0:
        return 

    try:
        shutil.copyfile(src, dst)
    except Exception as e:
        print(f"Broken file", src)

for fname in fnametrain:
    Calc(os.path.join(cat_directory, fnametrain), os.path.join(train_cats_directory, fname))
for fname in fnametest:
    Calc(os.path.join(cat_directory, fnametest), os.path.join(test_cats_directory, fname))

for fname in fnameval:
    Calc(os.path.join(cat_directory, fnameval), os.path.join(validation_cats_directory, fname))


for fname in fnametrain:
    Calc(os.path.join(dog_directory, fnametrain), os.path.join(train_dogs_directory, fname))
for fname in fnametest:
    Calc(os.path.join(dog_directory, fnametest), os.path.join(test_dogs_directory, fname))

for fname in fnameval:
    Calc(os.path.join(dog_directory, fnameval), os.path.join(validation_dogs_directory, fname))